In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.hadoop.fs.{FileSystem, Path}

In [2]:
val spark = SparkSession.builder()
      .appName("CaseStudy5 - Time-Based Data Partitioning for Ratings")
      .getOrCreate()

spark = org.apache.spark.sql.SparkSession@5c0a40e6


org.apache.spark.sql.SparkSession@5c0a40e6

In [3]:
val ratingsDataPath = "gs://spark-tasks-bucket/day_16_17/rating.csv"
val ratingsDF = spark.read.option("header", "true").option("inferSchema", "true").csv(ratingsDataPath)

ratingsDataPath = gs://spark-tasks-bucket/day_16_17/rating.csv
ratingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [4]:
ratingsDF.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [5]:
ratingsDF.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|   4.0|2004-09-10 03:08:54|
|     1|    223|   4.0|2005-04-02 23:46:13|
|     1|    253|   4.0|2005-04-02 23:35:40|
|     1|    260|   4.0|2005-04-02 23:33:46|
|     1|    293|   4.0|2005-04-02 23:31:43|
|     1|    296|   4.0|2005-04-02 23:32:47|
|     1|    318|   4.0|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|   4.0|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|   3.0|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

In [7]:
val updatedDF = ratingsDF.withColumn("year", year(col("timestamp")))

updatedDF = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [8]:
updatedDF.show()

+------+-------+------+-------------------+----+
|userId|movieId|rating|          timestamp|year|
+------+-------+------+-------------------+----+
|     1|      2|   3.5|2005-04-02 23:53:47|2005|
|     1|     29|   3.5|2005-04-02 23:31:16|2005|
|     1|     32|   3.5|2005-04-02 23:33:39|2005|
|     1|     47|   3.5|2005-04-02 23:32:07|2005|
|     1|     50|   3.5|2005-04-02 23:29:40|2005|
|     1|    112|   3.5|2004-09-10 03:09:00|2004|
|     1|    151|   4.0|2004-09-10 03:08:54|2004|
|     1|    223|   4.0|2005-04-02 23:46:13|2005|
|     1|    253|   4.0|2005-04-02 23:35:40|2005|
|     1|    260|   4.0|2005-04-02 23:33:46|2005|
|     1|    293|   4.0|2005-04-02 23:31:43|2005|
|     1|    296|   4.0|2005-04-02 23:32:47|2005|
|     1|    318|   4.0|2005-04-02 23:33:18|2005|
|     1|    337|   3.5|2004-09-10 03:08:29|2004|
|     1|    367|   3.5|2005-04-02 23:53:00|2005|
|     1|    541|   4.0|2005-04-02 23:30:03|2005|
|     1|    589|   3.5|2005-04-02 23:45:57|2005|
|     1|    593|   3

In [ ]:
// Has lot of records, so taking only 100000 records
val trimmedDF = updatedDF.limit(100000)

trimmedDF = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [11]:
val outputPath = "hdfs:///user/day_16_17/case_study_5"
trimmedDF.coalesce(1).write.partitionBy("year").format("parquet").mode("overwrite").save(outputPath)

outputPath = hdfs:///user/day_16_17/case_study_5


hdfs:///user/day_16_17/case_study_5

In [12]:
// Try reading the data from HDFS

val path = "hdfs:///user/day_16_17/case_study_5/year=1996"
spark.read.option("basePath", outputPath).parquet(path)
    .select("year").distinct().show()

// This suggests that the data is partitioned based on userId as expected

+----+
|year|
+----+
|1996|
+----+



path = hdfs:///user/day_16_17/case_study_5/year=1996


hdfs:///user/day_16_17/case_study_5/year=1996

In [13]:
spark.stop()